In [68]:
import pandas as pd
import json
from difflib import SequenceMatcher

### Part 1: Read a json file of outputs as identified by the classifier

In [114]:
#name = 'roberta_lg_mgn_2'
name = 'elmo_best'
with open("../../predictions/train_set_predictions/10_spacy_basic_large_movies_format_1_TRAIN.json") as json_file:
    all_predictions = json.load(json_file)

In [113]:
all_predictionss = dict()

for key in all_predictions.keys():
    if key not in all_predictionss.keys():
        all_predictionss[key] = dict()
    if 'positive_keywords' not in all_predictionss[key].keys():
        all_predictionss[key]['positive_keywords'] = list()
    all_predictionss[key]['positive_keywords'] = all_predictions[key][0]

KeyError: 0

In [115]:
def split_keyphrases(dict):
    for key in dict.keys():
        if "positive_keywords" in dict[key].keys():
            tmp_pos_keys = []
            #dict[key]["positive_keywords"] = dict[key]["positive_keywords"].split(" ")
            for keyphrase in dict[key]["positive_keywords"]:
                keywords = keyphrase.split(" ")
                tmp_pos_keys.extend(keywords)
            dict[key]["positive_keywords"] = list(set(tmp_pos_keys))
        if "negative_keywords" in dict[key].keys():
            tmp_neg_keys = []
            #dict[key]["negative_keywords"] = dict[key]["negative_keywords"].split(" ")
            for keyphrase in dict[key]["negative_keywords"]:
                keywords = keyphrase.split(" ")
                tmp_neg_keys.extend(keywords)
            dict[key]["negative_keywords"] = list(set(tmp_neg_keys))
    return dict

all_predictions = split_keyphrases(all_predictions)

In [116]:
# Fix completely wrong predictions, where 'NEW LINE' was returned as a movie

for key in all_predictions:
    if 'positive_movies' in all_predictions[key].keys():
        all_predictions[key]["positive_movies"] = [item for item in all_predictions[key]["positive_movies"] if item not in ['/', "The", 'of' "NEW_LINE", '.']]
        if len(all_predictions[key]["positive_movies"]) < 1:
            all_predictions[key].pop('positive_movies')
            
    if 'negative_movies' in all_predictions[key].keys():
        all_predictions[key]["negative_movies"] = [item for item in all_predictions[key]["negative_movies"] if item not in ["/", "The", 'of', "NEW_LINE", "."]]
        if len(all_predictions[key]["negative_movies"]) < 1:
            all_predictions[key].pop('negative_movies')
               
    if 'positive_keywords' in all_predictions[key].keys():
        all_predictions[key]["positive_keywords"] = [item for item in all_predictions[key]["positive_keywords"] if item not in ["/", "NEW_LINE", "."]]
        if len(all_predictions[key]["positive_keywords"]) < 1:
            all_predictions[key].pop('positive_keywords')
            
    if 'negative_keywords' in all_predictions[key].keys():
        all_predictions[key]["negative_keywords"] = [item for item in all_predictions[key]["negative_keywords"] if item not in ["/", "NEW_LINE", "."]]
        if len(all_predictions[key]["negative_keywords"]) < 1:
            all_predictions[key].pop('negativee_keywords')
            
    if 'positive_keywords' in all_predictions[key].keys():
        for i in range(0, len(all_predictions[key]["positive_keywords"])):
            all_predictions[key]["positive_keywords"][i] = all_predictions[key]["positive_keywords"][i].replace("/", "")   
    
    if 'negative_keywords' in all_predictions[key].keys():
        for i in range(0, len(all_predictions[key]["negative_keywords"])):
            all_predictions[key]["negative_keywords"][i] = all_predictions[key]["negative_keywords"][i].replace("/", "")   

### Part 2: Turn the original predictions to format 2 and save them as json

In [117]:
# Change the format of the predictions to fit recommender engine

all_predictions_2 = dict()

for key, value in all_predictions.items():
    if not key in all_predictions_2.keys():
        all_predictions_2[key] = list()
        for k in value.keys():
            if k=='positive_movies':
                for v in value[k]:
                    all_predictions_2[key].append("-mp " + "\"" + v + "\"")
            elif k=='negative_movies':
                for v in value[k]:
                    all_predictions_2[key].append("-mn " + "\"" + v + "\"")
            elif k=='positive_genres':
                for v in value[k]:
                    all_predictions_2[key].append("-gp " + "\"" + v + "\"")
            elif k=='negative_genres':
                for v in value[k]:
                    all_predictions_2[key].append("-gn " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    all_predictions_2[key].append("-amp " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    all_predictions_2[key].append("-amn " + "\"" + v + "\"")
            elif k=='positive_keywords':
                for v in value[k]:
                    all_predictions_2[key].append("-kp " + "\"" + v + "\"")
            elif k=='negative_keywords':
                for v in value[k]:
                    all_predictions_2[key].append("-kn " + "\"" + v + "\"")

In [118]:
with open("../../predictions/train_set_predictions/10_spacy_basic_large_movies_unmatched_format_2_TRAIN.json", "w") as outfile:
    json.dump(all_predictions_2, outfile)

In [119]:
# Dictionary of matching movies in several languages 
def create_alt_movie_dict(movies_matched):
    alt_names = dict()
    for index, row in movies_matched.iterrows():
        if row["original_title"] not in alt_names.keys():
            alt_names[row["original_title"]] = list()
        alt_names[row["original_title"]].append(row["alternative"])
    return alt_names

# Similarity function
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [120]:
# Read additional data
imdb_genres = pd.read_csv("D:/TU_Graz/Thesis/Datasets/IMDB_database/genres.csv", sep=';') # IMDB genre list to compare identified genres to
movie_titles = pd.read_csv("D:/TU_Graz/Thesis/Datasets/Reddit_original/movie_titles.csv", sep=';', encoding = 'latin')
movies_matched = pd.read_csv("D:/TU_Graz/Thesis/Datasets/IMDB_database/movies_matched.csv")
submissions = pd.read_csv("D:/TU_Graz/Thesis/Datasets/Reddit_original/submissions.csv", sep=';')
submissions = submissions.fillna("")

alt_names = create_alt_movie_dict(movies_matched)

In [121]:
all_predictions

{'10gvsh': {'positive_movies': ['Brick']},
 '10tqw6': {},
 '10u1ce': {},
 '1106mj': {},
 '11fhdf': {},
 '11h2xa': {},
 '11ms2h': {},
 '11nniy': {},
 '11oxif': {},
 '11prhd': {'positive_movies': ['Brotherhood of War    Ca', 'Oldboy']},
 '11tltq': {},
 '11ycep': {},
 '11zedd': {'positive_movies': ['Netflix']},
 '120ww0': {},
 '124iqq': {'positive_movies': ['The Man from Nowhere',
   'Battle Royale   Bloody']},
 '126xej': {},
 '128z5l': {},
 '12aufe': {},
 '12ifq2': {},
 '12j985': {},
 '12ko6p': {},
 '12r9wx': {},
 '12rll1': {'positive_movies': ['Saw Shutter Island']},
 '12sy3w': {},
 '12uukn': {},
 '12xb6d': {},
 '1300ha': {},
 '131xox': {'positive_movies': ['Visioneers The Invention of Lying action Blade Runner']},
 '139jol': {},
 '13dwdh': {'positive_movies': ['Eternal Sunshine of the Spotless Mind']},
 '13f82g': {'positive_movies': ['Spy Games Mission Impossible']},
 '13imzy': {},
 '13k6hq': {'positive_movies': ['Fight Club', 'Fight Club']},
 '13lq00': {},
 '13nmxq': {},
 '13pgkr': {}

In [45]:
movie = 'Island'
most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:-7]) >= 0.95) or similar('The ' + movie, mov[0:-7]) >= 0.95)]
mov = 'The Lord of the Ringispils: The Fellowship of the Ring (2001)'
mov[0:-7]
similar('Island', 'The Island')
most_similar

['The Island (2005)']

In [46]:
most_similar_alt = []
most_similar_alt_id = []
for k in alt_names.keys():
    for mov in alt_names[k]:
        if similar(movie, mov[0:len(movie)]) >= 0.8:
            most_similar_alt.append(k)
            most_similar_alt.append(mov)
                
most_similar_alt = list(dict.fromkeys(most_similar_alt))
if len(most_similar_alt) > 0:
    most_similar_alt_id = []
    for title in most_similar_alt:
        if title in movie_titles.movie_title.values:
            most_similar_alt_id.append(movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0])
            most_similar_alt_id = list(dict.fromkeys(most_similar_alt_id))

In [47]:
most_similar_alt

['¿Quién puede matar a un niño? (1976)',
 'Island of Death (1980)',
 'Island of the Damned (1978)',
 'Zombi Holocaust (1980)',
 'Island of the Last Zombies (1980)',
 'Zombi 2 (1979)',
 'Island of the Living Dead (1979)',
 'Island of the Flesh-Eaters (1979)',
 'Hrútar (2015)',
 'Islandzka opowiesc (2015)',
 'Ilha das Flores (1989)',
 'Island of Flowers (1989)',
 'Inland Empire (2006)',
 'Inland Empire (2007)',
 "Inland Empire - L'impero della mente (2006)",
 'Inland Empire - Eine Frau in Schwierigkeiten (2007)',
 'Inland Empire: A Woman in Trouble (2006)']

In [48]:
for genre_imdb in imdb_genres['genrename']:
    print(genre_imdb)

Action
Adult
Adventure
Animation
Biography
Comedy
Commercial
Crime
Documentary
Drama
Erotica
Experimental
Family
Fantasy
Film-Noir
Game-Show
Hardcore
History
Horror
Lifestyle
Music
Musical
Mystery
News
Reality-TV
Romance
Sci-Fi
Sex
Short
Sport
Talk-Show
Thriller
War
Western


### Part 3: Perform the matching

In [ ]:
imdb_predictions = dict()
for key in all_predictions.keys():
    imdb_predictions[key] = dict()
    for category in all_predictions[key].keys():
        if category not in imdb_predictions[key].keys():
            imdb_predictions[key][category] = []
            
        if category in ['positive_keywords', 'negative_keywords']:
            imdb_predictions[key][category] = all_predictions[key][category]
        
        elif category in ['positive_genres', 'negative_genres']:
            for genre in all_predictions[key][category]:
                for genre_imdb in imdb_genres.genrename.values:
                    if genre == genre_imdb:
                        imdb_predictions[key][category].append(genre_imdb)
                    else:
                        if similar(genre, genre_imdb) > 0.70:
                            imdb_predictions[key][category].append(genre_imdb)
            
            all_predictions[key][category] = list(set(all_predictions[key][category]))
            
        elif category in ['positive_actors', 'negative_actors']:
            imdb_predictions[key][category] = all_predictions[key][category]
        
        elif category in ['positive_movies', 'negative_movies']:
            for movie in all_predictions[key][category]:
                most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:-7]) >= 0.9) or similar('The ' + movie, mov[0:-7]) >= 0.9)]
                #most_similar = [mov for mov in movie_titles.movie_title.values if ((similar(movie, mov[0:len(movie)]) >= 0.8) or similar('The' + movie, mov[0:len(movie)+3]) >= 0.8)]

                if len(most_similar) > 0:
                    most_similar_id = [movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0] for title in most_similar]
                    most_similar_id = list(dict.fromkeys(most_similar_id))
                    #print(most_similar)
                    #print(most_similar_id)
                    imdb_predictions[key][category].extend(most_similar_id)
                else:
                    most_similar_alt = []
                    for k in alt_names.keys():
                        for mov in alt_names[k]:
                            if similar(movie, mov[0:-7]) >= 0.8:
                                most_similar_alt.append(k)
                                most_similar_alt.append(mov)
                
                    most_similar_alt = list(dict.fromkeys(most_similar_alt))
                    if len(most_similar_alt) > 0:
                        most_similar_alt_id = []
                        for title in most_similar_alt:
                            if title in movie_titles.movie_title.values:
                                most_similar_alt_id.append(movie_titles[movie_titles["movie_title"]==title]["movie_id"].values[0])
                        most_similar_alt_id = list(dict.fromkeys(most_similar_alt_id))
                        imdb_predictions[key][category].extend(most_similar_alt_id)
            
        #        for movie_database in movie_titles.movie_title
        

In [51]:
all_predictions_matched = imdb_predictions

In [ ]:
with open("../../predictions/train_set_predictions/10_spacy_basic_large_movies_matched_format_1_TRAIN.json", "w") as outfile:
    json.dump(imdb_predictions, outfile)

In [ ]:
imdb_predictions['5gchx2']

In [ ]:
imdb_predictions_2 = dict()
imdb_predictions = all_models

for key, value in imdb_predictions.items():
    if not key in imdb_predictions_2.keys():
        imdb_predictions_2[key] = list()
        for k in value.keys():
            if k=='positive_movies':
                for v in value[k]:
                    imdb_predictions_2[key].append("-mp " + "\"" + v + "\"")
            elif k=='negative_movies':
                for v in value[k]:
                    imdb_predictions_2[key].append("-mn " + "\"" + v + "\"")
            elif k=='positive_genres':
                for v in value[k]:
                    imdb_predictions_2[key].append("-gp " + "\"" + v + "\"")
            elif k=='negative_genres':
                for v in value[k]:
                    imdb_predictions_2[key].append("-gn " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    imdb_predictions_2[key].append("-amp " + "\"" + v + "\"")
            elif k=='positive_actors':
                for v in value[k]:
                    imdb_predictions_2[key].append("-amn " + "\"" + v + "\"")
            elif k=='positive_keywords':
                for v in value[k]:
                    imdb_predictions_2[key].append("-kp " + "\"" + v + "\"")
            elif k=='negative_keywords':
                for v in value[k]:
                    imdb_predictions_2[key].append("-kn " + "\"" + v + "\"")

In [ ]:
with open("../../predictions/train_set_predictions/10_spacy_basic_large_movies_matched_format_2_TRAIN.json", "w") as outfile:
    json.dump(imdb_predictions_2, outfile)